# Project description

The goal of this project is to build a database of the 10 000 most popular feature films on IMDB. We'll define "popularity" as the number of ratings a film has received. 

The project will comprise the following steps:

- we'll collect the data through scraping and API requests;
- we'll load the data onto a PostgreSQL database;
- we'll query the data with SQL;
- we'll visualize the results of the queries

# Modules

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time

# Data collection

IMDB doesn't have an official API. However, there are two unofficial APIs providing access to IMDB data: The Open Movie Database (OMDb) and The Movie Database (TMDB). They are both valid though OMDb provides more pieces of information and more accurate rating data. Unfortunately, OMDB enforces a strict rate limit of 1000 requests per day. We will thus rely on TMDB and integrate it with additional scraped data.

The TMDB API works by supplying the ID of the film(s) whose info we'd like to retrieve. The IDs consist of a double "t" followed by seven or eight digits. They can be found inside the film URL. For example, "tt0050782" is the ID for this film: https://www.imdb.com/title/tt0050782/. 

The first step will thus be retrieving all the IDs of the 10 000 most popular films on IMDB, which can be found at this [link](https://www.imdb.com/search/keyword/?mode=detail&page=1&title_type=movie&ref_=kw_ref_rt_vt&num_votes=5000%2C&sort=num_votes,desc).

## Scraping the film IDs and the URLs

To scrape the film IDs of our 10 000 films we'll need to iterate a for loop over the first 200 pages in the playlist (each containing 50 films). The film ID is the value of the 'href' attribute inside h3 elements with the "lister-item-header" class. We'll write a simple regex to extract the film ID. We'll also scrape the entire URL: it's going to come in handy later when we do the scraping.

In [56]:
film_ids = []
film_urls = []

for i in range(1, 201):
  
  content = requests.get("https://www.imdb.com/search/keyword/?mode=detail&page=" + str(i) + "&title_type=movie&ref_=kw_ref_rt_vt&num_votes=5000%2C&sort=num_votes,desc").content
  
  soup = BeautifulSoup(content)

  for film in soup.find_all("h3", class_="lister-item-header"):

    link = film.find("a").get("href")
    film_id = re.findall(pattern = r"tt\d+", string = link)[0]
    film_ids.append(film_id)

    film_url = "https://www.imdb.com" + link
    film_urls.append(film_url)
    

## API calls

We'll now write a function to request the data from TMDB. The function takes a vector of film IDs and for each ID it sends a GET request to the API. If the status code is equal to 200 (i.e. the request has been successful) we'll append the data to a Pandas dataframe. Otherwise, we'll just append the film_id and we'll fill the remaining columns with null values.

In [ ]:
def build_film_df(film_ids):

  df = pd.DataFrame(columns = ["id", "title", "release_date", "runtime", "country", "language", "genre", "studios", "budget", "revenue"])

  for film_id in film_ids:
      
      response = requests.get("https://api.themoviedb.org/3/movie/" + film_id + "?api_key=eb74f269207d7563d10da8190e228f61")

      if response.status_code == 200:  

        response_json = response.json()

        df = df.append({"id": response_json["imdb_id"],
                      "title": response_json["title"],
                      "release_date": response_json["release_date"],
                      "runtime": response_json["runtime"],
                      "country": ';'.join([country['name'] for country in response_json["production_countries"]]),
                      "language": ';'.join([language["english_name"] for language in response_json["spoken_languages"]]),
                      "genre": ';'.join([genre["name"] for genre in response_json["genres"]]),
                      "studios": ';'.join([company["name"] for company in response_json['production_companies']]),
                      "budget": response_json['budget'],
                      "revenue": response_json["revenue"]}, 
                       ignore_index = True)
        
      else:
        df = df.append({"id": film_id}, ignore_index = True)

  return df

In [ ]:
df = build_film_df(film_ids)
df

,id,title,release_date,runtime,country,language,genre,studios,budget,revenue,imdb_rating,imdb_votes
0,tt0111161,The Shawshank Redemption,1994-09-23,142,United States of America,English,Drama;Crime,Castle Rock Entertainment,25000000,28341469,8.7,20286
1,tt0468569,The Dark Knight,2008-07-14,152,United Kingdom;United States of America,English;Mandarin,Drama;Action;Crime;Thriller,DC Comics;Legendary Pictures;Syncopy;Isobel Gr...,185000000,1004558444,8.5,26365
2,tt1375666,Inception,2010-07-15,148,United Kingdom;United States of America,English;Japanese,Action;Science Fiction;Adventure,Legendary Pictures;Syncopy;Warner Bros. Pictures,160000000,825532764,8.4,30434
3,tt0137523,Fight Club,1999-10-15,139,Germany;United States of America,English,Drama,Regency Enterprises;Fox 2000 Pictures;Taurus F...,63000000,100853753,8.4,23013
4,tt0109830,Forrest Gump,1994-07-06,142,United States of America,English,Comedy;Drama;Romance,Paramount;The Steve Tisch Company,55000000,677387716,8.5,21749
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,tt0102900,Shattered,1991-10-11,98,United States of America,English,Mystery;Thriller,Palace Pictures;Metro-Goldwyn-Mayer,22000000,11511031,6.3,104
9996,tt0470833,Keloglan Vs. The Dark Prince,2006-01-06,90,Turkey,Turkish,Comedy,Energy Prodüksiyon,0,5000000,1.9,18
9997,tt0120238,SubUrbia,1996-02-07,121,United States of America,English,Comedy;Drama,Detour Filmproduction;Castle Rock Entertainment,0,656747,6.4,79
9998,tt0088748,Asterix vs. Caesar,1985-03-06,79,France,French,Comedy;Animation;Family;Adventure,Gaumont International;Dargaud Films;Les Produc...,0,0,6.4,413


Let's have a look at the null values

In [ ]:
df[df['title'].isnull()]

,id,title,release_date,runtime,country,language,genre,studios,budget,revenue,imdb_rating,imdb_votes
5240,tt9573980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9127,tt8075016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9976,tt13423846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Scraping additional data

TMDB doesn't provide much data about the people who worked on a film (directors, writer, actors etc.) and its rating data is inaccurate. For these reason, we'll integrate its data by scraping some more information.

### Custom functions

To implement sound software engineering principles we'll do the scraping by building a custom function for each type of data we need.

#### Film ID

In [49]:
def scrape_film_id(soup):
  
  return soup.find("meta", {"property": "imdb:pageConst"}).get("content")

#### Directors

In [3]:
def scrape_director(soup):
  
  directors = list(set([a.text for a in soup.find_all(href = re.compile("tt_ov_dr"), class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")]))

  return ';'.join(directors)



#### IMDB average rating

In [5]:
def scrape_imdb_rating(soup):
  
  pattern = r'(?<="ratingValue":)[\d.]+(?=},"contentRating")'
  string = str(soup.find("script", {"type": "application/ld+json"}))

  return re.findall(pattern = pattern,
                    string = string)[0]

IMDB rating count

In [6]:
def scrape_rating_count(soup):

  pattern = r'(?<="ratingCount":)[\d.]+'
  string = str(soup.find("script", {"type": "application/ld+json"}))

  return re.findall(pattern = pattern,
                    string = string)

#### Metascore

In [7]:
def scrape_metascore(soup):

  return soup.find("span", class_="score-meta").text

#### User reviews count

In [17]:
def scrape_user_review_count(soup):
  
  pattern = r'(?<="total":)\d+(?=,"__typename":"ReviewsConnection"})'
  
  string = str(soup.find("script", {'id': '__NEXT_DATA__'}))
  
  return re.findall(pattern = pattern, string = string)[0]

### Scraping

In [68]:
def build_scraped_df(urls):

  df = pd.DataFrame(columns = ["id", "director", "imdb_rating", "imdb_rating_count", "metascore", "user_review_count", "critic_review_count"])

  chunks = np.array_split(urls, 10)

  for chunk in chunks:
    for url in chunk:
      content = requests.get(url).content
      soup = BeautifulSoup(content)

      df = df.append({"id": scrape_film_id,
                      "director": scrape_director,
                      "imdb_rating": scrape_imdb_rating,
                      "imdb_rating_count": scrape_rating_count,
                      "metascore": scrape_metascore,
                      "user_review_count": scrape_user_review_count,
                      "critic_review_count": scrape_critic_review_count}, 
                     ignore_index = True)

    time.sleep(60*5)

  return df


In [69]:
df2 = build_scraped_df(film_urls)
df2.head()

NameError: ignored

#### Critic reviews count

In [33]:
def scrape_critic_review_count(soup):

  pattern = r'(?<=n"},"criticReviewsTotal":{"total":)\d+'
  
  string = str(soup.find("script", {'id': '__NEXT_DATA__'}))
  
  return re.findall(pattern = pattern, string = string)[-1]

#### Full cast

In [ ]:
def scrape_actors(soup):
  
  actors = [img.get("alt") for img in soup.find("table", class_="cast_list").find_all("img")]
  return ';'.join(actors)

# Load the data onto a database